# GLM-4-9B-Chat微调实战-LoRA技术微调

## 步骤1 导入相关包

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 配置环境变量
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# os.environ['HF_HOME'] = '/root/autodl-tmp/cache/huggingface'
# os.environ['MODELSCOPE_CACHE']='/root/autodl-tmp/cache/modelscope'

# 模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('ZhipuAI/glm-4-9b-chat')

# 设置关键参数
dataset_name_or_path = "datasets/identity_zh.json"
model_name_or_path = "models/ZhipuAI/glm-4-9b-chat"
train_output_dir = "saves/lora/glm-4-9b-chat"

## 步骤2 加载数据集

In [2]:
df = pd.read_json(dataset_name_or_path)
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 91
})

In [3]:
ds[:1]

{'instruction': ['hi'],
 'input': [''],
 'output': ['Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?']}

## 步骤3 数据集预处理

In [4]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
# 将tokenizer的pad_token设置为eos_token，这样在进行填充时，会使用eos_token作为填充符号
tokenizer.pad_token = tokenizer.eos_token
tokenizer

ChatGLM4Tokenizer(name_or_path='models/ZhipuAI/glm-4-9b-chat', vocab_size=151329, model_max_length=128000, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '[MASK]', '[gMASK]', '[sMASK]', '<sop>', '<eop>', '<|system|>', '<|user|>', '<|assistant|>', '<|observation|>', '<|begin_of_image|>', '<|end_of_image|>', '<|begin_of_video|>', '<|end_of_video|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151329: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151330: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151331: AddedToken("[gMASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151332: AddedToken("[sMASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	

补充说明：
- GLM4-9B-chat 采用的Prompt Template格式
``` text
[gMASK]<sop><|system|>\nSYSTEM_PROMPT<|user|>\nHUMAN_PROMPT<|assistant|>\nAI_OUT<|endoftext|>
```
- 例如
```text
[gMASK]<sop><|system|> 
假设你是皇帝身边的女人--甄嬛。<|user|> 
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|assistant|> 
嘘——都说许愿说破是不灵的。<|endoftext|>
```

In [5]:
def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], [] 
    instruction = tokenizer((f"[gMASK]<sop><|user|>\n{example['instruction']+example['input']}<|assistant|>\n"), 
                            add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)          # \n output <|endoftext|>
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
}

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 91
})

In [7]:
# 检查input_ids的格式
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK] <sop> <|user|> \nhello <|assistant|> \nHello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today? <|endoftext|>'

In [8]:
# 检查labels数据格式
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today? <|endoftext|>'

## 步骤4 创建模型

In [9]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, 
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16, 
                                             trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [10]:
 # 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()

In [11]:
#  查看数据类型
model.dtype 

torch.bfloat16

### 1、PEFT 步骤1 配置文件

In [12]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'dense', 'dense_h_to_4h', 'query_key_value', 'dense_4h_to_h'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### 2、PEFT 步骤2 创建模型

In [13]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(151552, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-39): 40 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False)
         

In [14]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

trainable params: 21,176,320 || all params: 9,421,127,680 || trainable%: 0.2248


## 步骤5 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir=train_output_dir,  # 输出目录，用于保存模型和日志
    per_device_train_batch_size=1,  # 每个设备的训练批次大小
    gradient_accumulation_steps=8,  # 梯度累积步数，用于模拟更大的批次大小
    logging_steps=50,  # 每隔多少步记录一次日志
    num_train_epochs=20,  # 训练的总轮数
    save_steps=100,  # 每隔多少步保存一次模型
    learning_rate=1e-5,  # 学习率
    save_on_each_node=True,  # 是否在每个节点上保存模型
    gradient_checkpointing=True  # 是否启用梯度检查点（减少内存占用）
)

## 步骤6 创建训练器

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
50,2.400100
100,0.685600
150,0.309200
200,0.114100


/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


TrainOutput(global_step=220, training_loss=0.8049717697230252, metrics={'train_runtime': 824.3796, 'train_samples_per_second': 2.208, 'train_steps_per_second': 0.267, 'total_flos': 3446242762162176.0, 'train_loss': 0.8049717697230252, 'epoch': 19.34065934065934})

## 步骤8 模型保存

In [18]:
# 保存模型及训练状态
model.save_pretrained(train_output_dir)

## 步骤9 模型推理
- <span style="background-color: yellow;">需要重启jupyter内核</span>

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import PeftModel

# 设置关键参数
model_name_or_path = "models/ZhipuAI/glm-4-9b-chat"
train_output_dir = "saves/lora/glm-4-9b-chat"

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# 加载Base模型
base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(base_model, model_id=train_output_dir)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
# 模型推理
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是谁？"}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


您好，我是 夜行者，由 木子 开发，旨在为用户提供智能化的回答和帮助。
